In [5]:
%load_ext autoreload
%autoreload 2

In [32]:
import json 
import numpy as np 
from arcs.setup_functions import ReactionGibbsandEquilibrium
from monty.serialization import loadfn

In [35]:
dft_data = loadfn('../app/data/hse06_dft_data.json')
rge = ReactionGibbsandEquilibrium(dft_data)
gibbs_free_energy = rge.Gibbs(compound='CO2',temperature=100,pressure=1)
gibbs_free_energy


-31.974842281859573

In [ ]:
reaction = 'CO + H2O = H2 + CO2'
from chempy import Equilibrium 
eq = Equilibrium.from_string(reaction)
rge.reaction_gibbs(reaction=eq,
                   pressure=1,
                   temperature=100)

TypeError: ReactionGibbsandEquilibrium.Gibbs() missing 2 required positional arguments: 'temperature' and 'pressure'